In [1]:
import requests
import bs4
from bs4 import BeautifulSoup 

In [2]:
def get_data(pageno,url):
    r=requests.get(url) 
    html_content=r.content 
    soup=BeautifulSoup(html_content)
    all=[]
    for d in soup.find_all('div',attrs={'class':'lister-item mode-advanced'}):
        all1=[]
        a_tags=d.find_all('a')
        movie_name=a_tags[1]
        if movie_name is not None:
            all1.append(movie_name.text)
        else:
            all1.append('unknown')
        
        rating=d.find('strong')
        if rating is not None:
            all1.append(rating.text)
        else:
            all1.append('unknown')
    
        p_tags=d.find_all('p')
        description=p_tags[1]
        if description is not None:
            all1.append(description.text)
        else:
            all1.append('unknown')  
        
        votes=d.find('span',attrs={'name':'nv'})
        if votes is not None:
            all1.append(votes.text)
        else:
            all1.append('unknown')
        
        
        directors=p_tags[2].find_all()
        all_directors=""
        flag=0
        length=0
        for j in range(len(directors)):
            span = directors[j].get('class')
            if span is not None:
                if span[0] == 'ghost':
                    flag=1
                    length=j
                    break
            all_directors+=directors[j].text + ", "
        if flag==1:
            all1.append(all_directors)
        else:
            all1.append('unknown')
    
        stars=""
        for k in range(length+1,len(directors)):
            stars+=directors[k].text + ", "
        if stars is not None:
            all1.append(stars)
        else:
            all1.append('unknown')
    
        score=d.find('div',attrs={'class':'inline-block ratings-metascore'})
        if score is not None:
            metascore=score.find('span')
            if metascore is not None:
                all1.append(metascore.text)
            else:
                all1.append('unknown')
        else:
            all1.append('unknown')
                
        time_duration=d.find('span',attrs={'class':'runtime'})
        if time_duration is not None:
            all1.append(time_duration.text)
        else:
            all1.append('unknown')
        
        genre=d.find('span',attrs={'class':'genre'})
        if genre is not None:
            all1.append(genre.text)
        else:
            all1.append('unknown')
        
        all.append(all1)
    return all

In [4]:
import pandas as pd
totalMovies = 1000
mvif=[]
no=1
i=1
while no<=1000:
    i=i+1
    if no==1:
        url=('https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&page=1')
    else:
        url=('https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start='+str(no)+'&ref_=adv_nxt')
    mvif.append(get_data(no,url))
    no=1+((i-1)*50)
flatten = lambda l: [item for sublist in l for item in sublist]
movies_df = pd.DataFrame(flatten(mvif), columns = ['Movie name', 'IMDB Rating','Description', 'Votes', 'Director(s)', 'Stars', 'Metascore','Total Duration','Movie Genre'])

movies_df

,Movie name,IMDB Rating,Description,Votes,Director(s),Stars,Metascore,Total Duration,Movie Genre
0,Tenet,7.4,"\nArmed with only one word, Tenet, and fightin...","435,495","Christopher Nolan,","John David Washington, Robert Pattinson, Eliza...",69,150 min,"\nAction, Sci-Fi, Thriller"
1,The Queen's Gambit,8.6,"\nOrphaned at the tender age of nine, prodigio...","364,550",unknown,"Chloe Pirrie, Bill Camp, Marcin Dorocinski,",unknown,395 min,\nDrama
2,Soul,8.1,"\nAfter landing the gig of a lifetime, a New Y...","284,836","Pete Docter, Kemp Powers,","Jamie Foxx, Tina Fey, Graham Norton, Rachel Ho...",83,100 min,"\nAnimation, Adventure, Comedy"
3,Wonder Woman 1984,5.4,\nDiana must contend with a work colleague and...,"237,701","Patty Jenkins,","Gal Gadot, Chris Pine, Kristen Wiig, Pedro Pas...",60,151 min,"\nAction, Adventure, Fantasy"
4,Birds of Prey: And the Fantabulous Emancipatio...,6.1,"\nAfter splitting with the Joker, Harley Quinn...","214,025","Cathy Yan,","Margot Robbie, Rosie Perez, Mary Elizabeth Win...",60,109 min,"\nAction, Adventure, Comedy"
...,...,...,...,...,...,...,...,...,...
995,Star Wars: The Clone Wars,8.5,\nThe Republic plans a daring final strike aga...,"3,276","Brian O'Connell, Dave Filoni,","Dee Bradley Baker, Matt Lanter, Matthew Wood, ...",unknown,23 min,"\nAnimation, Action, Adventure"
996,Bad Hair,5.6,\nIn 1989 an ambitious young woman gets a weav...,"3,275","Justin Simien,","Zaria Kelley, Corinne Massiah, Elle Lorraine, ...",61,102 min,"\nComedy, Horror, Music"
997,Veneno,8.7,"\nA biographical story of La Veneno, transgend...","3,272",unknown,"Isabel Torres, Paca la Piraña, Daniela Santiago,",unknown,421 min,"\nBiography, Drama"
998,The Undoing,7.9,\nGrace Fraser becomes intrigued by a young mo...,"3,271","Susanne Bier,","Nicole Kidman, Hugh Grant, Edgar Ramírez, Noah...",unknown,54 min,"\nCrime, Drama, Mystery"
